# Evaluating RAG w/ Alpha Tuning
Evaluation is a crucial piece of development when building a RAG pipeline. Likewise, alpha tuning can be a time consuming exercise to build out, so what does the performance benefit look like for all of this extra effort? Let's dig into that.

### Fixtures
- For our dataset: Llama2 Paper
- Our vector db: [Pinecone](https://www.pinecone.io/)
- For our embedding model: [ada-002](https://platform.openai.com/docs/models/embeddings) 
- For our LLM: [GPT-3.5 Turbo](https://platform.openai.com/docs/models/gpt-3-5-turbo)

These fixtures were chosen because they're well integrated within LlamaIndex to make this notebook very transferrable to those looking to reproduce this.
Likewise, Pinecone supports hybrid search, which is a requirement for hybrid searches to be possible. Koda Retriever will also be used!

### Testing

Koda Retriever was largely inspired from the alpha tuning [blog post written by Ravi Theja](https://blog.llamaindex.ai/llamaindex-enhancing-retrieval-performance-with-alpha-tuning-in-hybrid-search-in-rag-135d0c9b8a00) from Llama Index. For that reason, we'll follow a similar pattern and evaluate with:
- MRR (Mean Reciprocal Rank)
- Hit Rate

### Agenda:
- Fixture Setup
- Data Ingestion
- Synthetic Query Generation
- Alpha Mining
- Evaluation

In [ ]:
# Import all the necessary modules
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.postprocessor import LLMRerank
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import Settings
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.packs.koda_retriever import KodaRetriever
import os
from pinecone import Pinecone

## Fixture Setup

In [ ]:
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
index = pc.Index("llama2-paper")  # this was previously created in my pinecone account

Settings.llm = OpenAI()
Settings.embed_model = OpenAIEmbedding()

# if you recreate this using the default dataset in pinecone, you'll want to set `text_key = "summary"`
vector_store = PineconeVectorStore(pinecone_index=index)
vector_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store, embed_model=Settings.embed_model
)

reranker = LLMRerank(llm=Settings.llm)  # optional

koda_retriever = KodaRetriever(
    index=vector_index,
    llm=Settings.llm,
    reranker=reranker,  # optional
    verbose=True,
)

vanilla_retriever = vector_index.as_retriever()